<a href="https://colab.research.google.com/github/Novadotgg/Crop-recommendation-chatbot/blob/main/crop_recommendation_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn==1.2.2
!pip install flask_ngrok
!pip install flask_session
!pip install pyngrok
from flask import Flask, request, jsonify, session, render_template, render_template_string
from flask_ngrok import run_with_ngrok
import joblib
import nltk
from nltk.tokenize import word_tokenize
import spacy
from flask_session import Session
!pip install flask-ngrok
!pip install flask-session
!pip install joblib
!pip install spacy
!python -m spacy download en_core_web_sm
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")
!ngrok authtoken 2jB3oS2xptgkYHLMldb5uNwjHEF_3k3Q4UuBthT1xD6nyitXA

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 21.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restar

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:
#final crop recommendation chatbot
import requests
import numpy as np
from flask import Flask, request, jsonify, render_template_string, session
import joblib
from flask_session import Session
from pyngrok import ngrok
from transformers import pipeline
import re

# Load the crop prediction model
crop_model = joblib.load('/content/adaboost_model_soil.pkl')

# Initialize NLP pipelines
intent_classifier = pipeline("text-classification", model="distilbert-base-uncased")
sentiment_analyzer = pipeline("sentiment-analysis")
response_generator = pipeline("text-generation", model="gpt2")

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# Initialize session management
app.config['SESSION_TYPE'] = 'filesystem'
Session(app)

def fetch_weather_data(location):
    api_key = '3b0880537a825f35ec5cc49574db842e'
    base_url = f'http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric'
    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()
        temperature = data['main']['temp']
        humidity = data['main']['humidity']
        rainfall = data.get('rain', {}).get('1h', 0)
        return temperature, humidity, rainfall
    else:
        return None, None, None

questions = [
    'Please provide the Nitrogen (N) value:',
    'Please provide the Phosphorous (P) value:',
    'Please provide the Potassium (K) value:',
    'Please provide the pH value:'
]

# NLP functions
def detect_intent(user_message):
    result = intent_classifier(user_message)
    intent = result[0]['label']
    confidence = result[0]['score']
    return intent, confidence

def analyze_sentiment(user_message):
    result = sentiment_analyzer(user_message)
    sentiment = result[0]['label']
    return sentiment

def generate_response(context):
    response = response_generator(context, max_length=50, num_return_sequences=1)
    return response[0]['generated_text']

def extract_parameters(user_message):
    values = {}
    match = re.search(r'n\s*=\s*(\d+)', user_message, re.IGNORECASE)
    if match:
        values['N'] = int(match.group(1))
    match = re.search(r'p\s*=\s*(\d+)', user_message, re.IGNORECASE)
    if match:
        values['P'] = int(match.group(1))
    match = re.search(r'k\s*=\s*(\d+)', user_message, re.IGNORECASE)
    if match:
        values['K'] = int(match.group(1))
    match = re.search(r'ph\s*=\s*(\d+\.?\d*)', user_message, re.IGNORECASE)
    if match:
        values['pH'] = float(match.group(1))
    return values

@app.route('/')
def index():
    return render_template_string('''<!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Crop Recommendation Chatbot</title>
        <style>
            body, html { height: 100%; margin: 0; display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; }
            #chat-container { display: flex; flex-direction: column; align-items: center; width: 100%; max-width: 600px; }
            #chat-box { width: 100%; height: 300px; border: 1px solid #ccc; padding: 10px; margin-bottom: 10px; overflow-y: auto; text-align: left; }
            #input-container { display: flex; width: 100%; margin-bottom: 10px; }
            #user-input { flex: 1; padding: 10px; border: 1px solid #ccc; border-radius: 100px; }
            button { padding: 10px 20px; margin-left: 10px; border-radius: 100px; cursor: pointer; border: none; background-color: #006400; color: white; }
            button:hover { background-color: #45a049; color: white; border-radius: 100px; }
            h1 { margin-bottom: 20px; text-align: center; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2); font-family: 'Times New Roman', Times, serif; }
        </style>
    </head>
    <body>
        <div id="chat-container">
            <h1>Crop Recommendation Chatbot</h1>
            <div id="chat-box"></div>
            <div id="input-container">
                <input type="text" id="user-input" placeholder="Type your message...">
                <button onclick="sendMessage()">Send</button>
            </div>
        </div>
        <script>
            async function sendMessage() {
                const userInput = document.getElementById('user-input').value;
                const chatBox = document.getElementById('chat-box');
                chatBox.innerHTML += `<p>User: ${userInput}</p>`;

                let response = await fetch('/chatbot', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ message: userInput })
                });
                response = await response.json();
                chatBox.innerHTML += `<p>Bot: ${response.reply}</p>`;
                chatBox.scrollTop = chatBox.scrollHeight;
                document.getElementById('user-input').value = '';
            }

            document.getElementById('user-input').addEventListener('keydown', (event) => {
                if (event.key === 'Enter') {
                    event.preventDefault();
                    sendMessage();
                }
            });
        </script>
    </body>
    </html>
    ''')

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_message = request.json['message'].lower()

    # Define synonyms for intent matching
    recommend_synonyms = ['recommend', 'suggest', 'advise', 'what crops can i grow', 'what should i plant', 'crop recommendation','yes','ya']
    positive_feedback = ['thanks', 'great', 'good', 'helpful', 'useful', 'love', 'like', 'happy', 'nic']
    negative_feedback = ['bad', 'not good', 'useless', 'hate', 'dislike', 'terrible', 'worst']

    # Reset intent if user wants to start a new recommendation
    if any(word in user_message for word in recommend_synonyms):
        session['intent'] = 'recommend_crop'
        session['step'] = 0
        session['data'] = []
        session['recommendations'] = []
        session['feedback_stage'] = False  # Reset feedback stage
        return jsonify({'reply': 'Please provide your location (city or area):'})

    # Feedback handling - check if we're in the feedback stage
    if session.get('feedback_stage', False):
        sentiment = analyze_sentiment(user_message)
        if any(word in user_message for word in negative_feedback) or sentiment == 'NEGATIVE':
            session['feedback_stage'] = False  # Reset feedback stage after feedback
            return jsonify({'reply': 'I understand your concerns. For now ask if you need any recommendatiion'})
        elif any(word in user_message for word in positive_feedback) or sentiment == 'POSITIVE':
            session['feedback_stage'] = False  # Reset feedback stage after feedback
            # Ask if the user wants more recommendations
            later= jsonify({'reply': 'I’m glad to hear that! <br><br>Would you like to get some more recommendations.'})
            return later
            if later == 'yes':
                return jsonify({'reply': 'Please provide your location (city or area):'})
                session['intent'] = 'recommend_crop'
                session['step'] = 0
                session['data'] = []
                session['recommendations'] = []
                session['feedback_stage'] = False  # Reset feedback stage
            else:
              if later == 'no':
                return jsonify({'reply': 'Thanks for your time'})
        else:
            return jsonify({'reply': "Thank you for your feedback. If you have more to share, please let me know."})

    # Location input handling
    if session.get('intent') == 'recommend_crop' and session['step'] == 0:
        session['location'] = user_message
        temperature, humidity, rainfall = fetch_weather_data(session['location'])

        if temperature is None or humidity is None or rainfall is None:
            return jsonify({'reply': 'Could not fetch weather data for the provided location. Please try again with a valid location.'})

        session['weather'] = {'temperature': temperature, 'humidity': humidity, 'rainfall': rainfall}
        session['step'] += 1
        return jsonify({'reply': questions[0]})

    # Step-by-step parameter collection
    if session.get('intent') == 'recommend_crop':
        step = session['step'] - 1
        session['data'].append(user_message)

        # Collecting parameters
        if step < len(questions) - 1:
            session['step'] += 1
            return jsonify({'reply': questions[session['step'] - 1]})
        else:
            # Prediction after collecting all data
            try:
                features = session['data'] + [
                    session['weather']['temperature'],
                    session['weather']['humidity'],
                    session['weather']['rainfall']
                ]
                features = np.array(features, dtype=float).reshape(1, -1)
                predictions = crop_model.predict(features)
                session['recommendations'] = list(predictions)
                session['feedback_stage'] = True  # Set feedback mode

                if session['recommendations']:
                    crops = ', '.join(session['recommendations'])
                    response = f'Based on the provided data, you can grow {crops}. <br><br>What do you think about this recommendation?'
                    return jsonify({'reply': response})
                else:
                    return jsonify({'reply': 'No suitable crops found based on your input.'})

            except Exception as e:
                return jsonify({'reply': 'Sorry, I could not process your request. Please try again.'})

    # Fallback response for unrecognized inputs
    return jsonify({'reply': "I'm here to help with crop recommendations! Please start by requesting a recommendation."})






config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:


if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f" * Running on {public_url}")
    app.run()

 * Running on NgrokTunnel: "https://6820-34-55-102-59.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:07:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:07:51] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:07:53] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:08:03] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:08:06] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:08:09] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:08:10] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:08:11] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:08:12] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 05:09:16] "POST /chatb